## Quantus: built-in XAI methods overview.
There are no XAI libraries focusing on NLP models, so we provide a range of popular XAI methods built-in in Quantus.
Some of them are just wrappers around 3-rd party libraries (e.g., LIME, SHAP, NoiseGrad).
They-re compatible with Torch and tensorflow models from Huggingface Hub out of the box.
In order to use them with your own model, you have to implement `TensorflowTextClassifier` or `TorchTextClassifier` respectively.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import quantus.nlp as qn
from tqdm.auto import tqdm
from quantus.nlp.helpers.utils import map_explanations

#### 1. Create model, generate predictions.

In [ ]:
model = qn.TensorFlowHuggingFaceTextClassifier.from_pretrained("gchhablani/bert-base-cased-finetuned-sst2", from_pt=True)

In [ ]:
x_batch = ["director rob marshall went out gunning to make a great one ."]
y_batch = model.predict(x_batch).argmax(axis=-1)
model.unwrap().config.id2label[y_batch[0]]

#### 2. Generate explanations
All available XAI methods can be listed with `quantus.nlp.available_xai_methods()`

In [ ]:
xai_methods = qn.available_xai_methods()
xai_methods.sort()
xai_methods

The main "entrypoint" for using built-in XAI methods is `quantus.nlp.explain`

In [ ]:
a_batch = []
pbar = tqdm(xai_methods)
for i in pbar:
    pbar.desc = i
    a = qn.explain(model, x_batch, y_batch, method=i)
    a_batch.append(a[0])

In [ ]:
a_batch = map_explanations(a_batch, qn.normalize_sum_to_1)

In [ ]:
qn.visualise_explanations_as_pyplot(a_batch, xai_methods)

Gradient-based XAI methods can be called as well with plain-text inputs, as well as directly with token embeddings.
In practise this means, that for robustness metrics they can be used in combination with plain-text perturbation functions (e.g., typo_replacement)
and numerical ones (e.g., gaussian_noise). To verify which XAI methods support passing token embeddings we can see output of `quantus.nlp.available_numerical_xai_methods()`

In [ ]:
qn.available_numerical_xai_methods()